# Ruled Based Model
### Hey Together,   
Welcome to the first notebook for the ruled based model of commfy

   
## Part 1 import libarys

In [6]:
import random
from random import randint


## Part 2 gether Input
there is the option to give input yourself, to auto generate it once or to use the function input_generator() whenever you need it.

In [8]:
## auto generated as a function to use later:
def input_generator():
    weather=randint(1, 4)
    temperature=round(random.uniform(-10,35),2)
    if weather==1: weather='sunny'
    if weather==2: weather='rainy'
    if weather==3: 
        weather='snowy'
        temperature=min(temperature,4.)
    if weather==4: weather='windy'   
    
    duration=randint(0,1)
    sex=randint(0,1)
    return weather,temperature,duration,sex
def print_input(list):
    print('the input for...\nweather is: ',list[0],'\ntemperature is: ',list[1],'\nduration is: ',list[2],'\nsex is: ',list[3])

In [3]:
# ask for weather, temperature, duration and the sex
weather=input('please describe the weather as sunny; rainy; snowy or windy :  ')
temperature=input('please insert the temperature :  ')
temperature=float(temperature)
duration=input('please insert if the duration is longer than 20 min=1 or less then 20 min =0 :  ')
sex=input('please tell me you sex as female=1 or not female=0 :  ')
print('your input for...\nweather is: ',weather,'\ntemperature is: ',temperature,'\nduration is: ',duration,'\nsex is: ',sex)
input_selfgenerated=(weather,temperature,duration,sex)

your input for...
weather is:  sunny 
temperature is:  10.0 
duration is:  0 
sex is:  0


In [4]:
print_input(input_selfgenerated)

the input for...
weather is:  sunny 
temperature is:  10.0 
duration is:  0 
sex is:  0


In [9]:
#test input generator
z=input_generator()
print(z, ' -->')
print_input(z)

('windy', 17.97, 0, 1)  -->
the input for...
weather is:  windy 
temperature is:  17.97 
duration is:  0 
sex is:  1


<hr style="border:2px solid gray"> </hr>   
   
## Part 3 define the model

In [10]:
## so far just temperature and weather are looked at
def get_clothing_output_rbm(list):
    if list[0]=='sunny':
      if list[1]>25.0 : return 'sunny_hot'
      if list[1]>15.0 : return 'sunny_casual'
      return 'sunny_cold'
    if list[0]=='rainy':
      if list[1]>15.0 : return 'rainy_warm'
      return 'rainy_cold'
    if list[0]=='snowy': return 'snowy'
    if list[0]=='windy':
      if list[1]>15: return 'windy_warm'
      return 'windy_cold'
    return 'not defined'

In [11]:
#test the model
for i in range(20):
  a=input_generator()
  print(i+1,'  generated input: ',a, '  -->   ',get_clothing_output_rbm(a))


1   generated input:  ('rainy', 5.85, 0, 0)   -->    rainy_cold
2   generated input:  ('sunny', -5.45, 0, 1)   -->    sunny_cold
3   generated input:  ('windy', -2.71, 1, 0)   -->    windy_cold
4   generated input:  ('windy', 23.11, 1, 0)   -->    windy_warm
5   generated input:  ('sunny', -9.34, 0, 1)   -->    sunny_cold
6   generated input:  ('rainy', 34.46, 1, 1)   -->    rainy_warm
7   generated input:  ('snowy', 4.0, 0, 0)   -->    snowy
8   generated input:  ('rainy', 11.27, 1, 1)   -->    rainy_cold
9   generated input:  ('windy', 28.11, 0, 0)   -->    windy_warm
10   generated input:  ('rainy', 26.79, 0, 0)   -->    rainy_warm
11   generated input:  ('rainy', -4.79, 0, 1)   -->    rainy_cold
12   generated input:  ('sunny', 9.4, 1, 1)   -->    sunny_cold
13   generated input:  ('sunny', 28.95, 1, 1)   -->    sunny_hot
14   generated input:  ('rainy', 32.11, 1, 1)   -->    rainy_warm
15   generated input:  ('rainy', 22.13, 1, 1)   -->    rainy_warm
16   generated input:  ('snowy

<hr style="border:2px solid gray"> </hr>  

## Part 4: output generating
todo: what does snowy, or sunny_hot mean. --> translate the string into a Json for the Flask API